In [1]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import os

In [2]:
main_folder = 'C:/D/dev/data/20171205_cirrus_manual_results'
subfolders = ['CC_0003', 'CC_0005', 'CC_0006_', 'CC_0008']
filename = 'deformed_layer_points.csv' 

filenames = []
for subfolder in subfolders:
    filepath = os.path.join(main_folder, subfolder)
    filepath = os.path.join(filepath, filename)
    if os.path.exists(filepath):
        filenames.append(filepath)
        
filenames

['C:/D/dev/data/20171205_cirrus_manual_results\\CC_0003\\deformed_layer_points.csv',
 'C:/D/dev/data/20171205_cirrus_manual_results\\CC_0005\\deformed_layer_points.csv',
 'C:/D/dev/data/20171205_cirrus_manual_results\\CC_0006_\\deformed_layer_points.csv',
 'C:/D/dev/data/20171205_cirrus_manual_results\\CC_0008\\deformed_layer_points.csv']

In [4]:
import numpy as np
import csv
def read_layer_points(filename, verbose= False, n_scanlines=512, n_slices=128):
  
    each_layer_points = np.zeros((n_scanlines*n_slices, 3), dtype=np.float32)
    layer_points = dict()
    with open(filename, 'r') as cvsfile:
        reader = csv.reader(cvsfile, delimiter=',')
        label, row_cnt = 0, 0
        for count, row in enumerate(reader):
            #if count > 65538: break
            #if count > 65534:
            #    print('Row > 65534: ', row)
            #    if label != row[0]:
            #        print('> 65534, Label ne row[0]:', row)
            if not row[0].isnumeric(): 
                print('Not numeric ', row)
                continue
            if row[0] == label: # existing label
                each_layer_points[row_cnt, :] = np.float32(row[1:])
                #if count > 65534:
                #    print('> 65534 Assigned ', each_layer_points[row_cnt, :])
                #print('Row: ', row, ' pnt: ', each_layer_points[row_cnt, :])    
            else: # new label
                if label != 0: # safe pnts associated with previous label and create new data storage
                    if verbose: print('Safe to Dict: Row ', row, 'Label ', label, ' end point ', each_layer_points[-1, :])
                    layer_points[label] = each_layer_points
                    each_layer_points = np.zeros((n_scanlines*n_slices, 3), dtype=np.float32)
                    row_cnt = 0
                #set new label and store a new point
                if label == 0: print('Label==0 Row: ', row, 'row cnt ', row_cnt, ' pnt ', np.float32(row[1:]))
                label = row[0]
                each_layer_points[row_cnt, :] = np.float32(row[1:])
                
            row_cnt+=1
            
    return layer_points

layer_points = read_layer_points(filenames[0], verbose=True)


In [ ]:
def get_atlas_layer_points(filenames, verbose = False, n_scanlines=512, n_slices=128):
    
    """
        filenames : a list of filename containing layer points
    """
    atlas_layer_points = dict()
    n_files = len(filenames)
    n_points_per_file =n_scanlines*n_slices
    n_points = n_files*n_points_per_file
    start_indx, end_indx = 0,0
    
    for file_cnt, filename in enumerate(filenames):
        print('File number ', file_cnt, ' filename ', filename)
        layer_points = read_layer_points(filename, verbose=verbose)
        
        for label, points in layer_points.items():
            #print('Label ', label, '\nPoints\n', points[:2, :])
            # if label not exist, allocate array for the label in dict
            if label not in atlas_layer_points:
                atlas_layer_points[label] = np.zeros((n_points, 3), dtype=np.float32)
                end_indx = n_points_per_file
                
            # assign point
            atlas_layer_points[label][start_indx:end_indx, :] = layer_points[label]
            print('Label ', label, '\nPoints\n', points[:2, :],'\nAssignedPnts: \n', atlas_layer_points[label][start_indx:start_indx+2, :])
            
        #debug
        print('-------------Finish one file---------------------\n')
        tmp_label = str(file_cnt+1)
        if file_cnt == 0:
            print('Label', tmp_label)
            print('Start \n', atlas_layer_points[tmp_label][start_indx:start_indx+2, :])
            print('End \n', atlas_layer_points[tmp_label][end_indx-2:end_indx, :])
        else:
            print('Label', tmp_label)
            print('Start \n', atlas_layer_points[tmp_label][start_indx-2:start_indx+2, :])
            print('End \n', atlas_layer_points[tmp_label][end_indx-2:end_indx, :])
        #update indices
        start_indx = end_indx
        end_indx += n_points_per_file
        #end_indx = (file_cnt+2)*n_points_per_file
        #start_indx = end_indx-n_points_per_file
        print('Start index ',start_indx, ' end index ', end_indx)
        
    return atlas_layer_points

In [5]:
read_layer_points(filenames[0])

{'1': array([[  8.05157006e-01,  -1.60218996e-03,  -3.55315000e-01],
        [  8.04476023e-01,   1.00966003e-02,  -3.55311990e-01],
        [  8.03830981e-01,   2.17955001e-02,  -3.55309010e-01],
        ..., 
        [  7.01961994e-01,   5.95178986e+00,   5.64078999e+00],
        [  7.01323986e-01,   5.96349001e+00,   5.64080000e+00],
        [  7.00846016e-01,   5.97519016e+00,   5.64080000e+00]], dtype=float32),
 '2': array([[  8.21524024e-01,  -1.57338998e-03,  -3.55298996e-01],
        [  8.20814013e-01,   1.01253996e-02,  -3.55295986e-01],
        [  8.20200980e-01,   2.18243003e-02,  -3.55293006e-01],
        ..., 
        [  7.96693981e-01,   5.95193005e+00,   5.64087009e+00],
        [  7.97429025e-01,   5.96363020e+00,   5.64087009e+00],
        [  7.98160017e-01,   5.97532988e+00,   5.64088011e+00]], dtype=float32),
 '3': array([[  8.66532981e-01,  -1.49418996e-03,  -3.55257004e-01],
        [  8.66042972e-01,   1.02049997e-02,  -3.55253994e-01],
        [  8.65388989e-01, 

In [4]:
atlas_layer_points = dict()

n_files,n_scanlines, n_slices = len(filenames),512,128

for filename in filenames:
    layer_points = read_layer_points(filename)
    for label, points in layer_points.items():
        # populate the dict of atlas layer points using the data from the 1st file
        if len(atlas_layer_points) < 9:
            atlas_layer_points[label] = points
        # 
        elif atlas_layer_points[label].shape[0] > 0:
            # inspect process 
            current_after_end_index = atlas_layer_points[label].shape[0]
            prev_end_pnts = atlas_layer_points[label][-3:,:]
            new_begin_pnts = points[:2,:]
            # concatenate the points
            prev_points = atlas_layer_points[label]
            points = np.vstack((prev_points, points))
            atlas_layer_points[label] = points
            print('Previous end points: {}\nNew begin points: {}'.format(prev_end_pnts, new_begin_pnts))
            print('After concatenate \n', atlas_layer_points[label][current_after_end_index-3:current_after_end_index+2,:])
        else:
            print('error')

Previous end points: [[ 0.992127    5.95226002  5.64104986]
 [ 0.99184698  5.96396017  5.64104986]
 [ 0.99146599  5.97565985  5.64104986]]
New begin points: [[ 0.99542999  0.0483492  -0.0115824 ]
 [ 0.99566901  0.060049   -0.0115738 ]]
After concatenate 
 [[ 0.992127    5.95226002  5.64104986]
 [ 0.99184698  5.96396017  5.64104986]
 [ 0.99146599  5.97565985  5.64104986]
 [ 0.99542999  0.0483492  -0.0115824 ]
 [ 0.99566901  0.060049   -0.0115738 ]]
Previous end points: [[ 0.83788002  5.95200014  5.64091015]
 [ 0.839472    5.96369982  5.64091015]
 [ 0.84086198  5.97539997  5.64092016]]
New begin points: [[ 0.84766799  0.0486179  -0.0115015 ]
 [ 0.84806901  0.0603177  -0.0114928 ]]
After concatenate 
 [[ 0.83788002  5.95200014  5.64091015]
 [ 0.839472    5.96369982  5.64091015]
 [ 0.84086198  5.97539997  5.64092016]
 [ 0.84766799  0.0486179  -0.0115015 ]
 [ 0.84806901  0.0603177  -0.0114928 ]]
Previous end points: [[ 1.02127004  5.95231009  5.6410799 ]
 [ 1.02135003  5.96401978  5.6410799

In [5]:
# number of points of each layers from all images
data_size = n_files*n_scanlines*n_slices
print('Number of points with label #1 ', atlas_layer_points['1'].shape[0], ' should be ', data_size)
atlas_layer_points

Number of points with label #1  262144  should be  262144


{'1': array([[  8.05157006e-01,  -1.60218996e-03,  -3.55315000e-01],
        [  8.04476023e-01,   1.00966003e-02,  -3.55311990e-01],
        [  8.03830981e-01,   2.17955001e-02,  -3.55309010e-01],
        ..., 
        [  7.10080981e-01,   5.97811985e+00,   5.83028984e+00],
        [  7.05187976e-01,   5.98982000e+00,   5.83028984e+00],
        [  7.04985023e-01,   6.00152016e+00,   5.83027983e+00]], dtype=float32),
 '2': array([[  8.21524024e-01,  -1.57338998e-03,  -3.55298996e-01],
        [  8.20814013e-01,   1.01253996e-02,  -3.55295986e-01],
        [  8.20200980e-01,   2.18243003e-02,  -3.55293006e-01],
        ..., 
        [  7.90126979e-01,   5.97806978e+00,   5.83027983e+00],
        [  7.84793973e-01,   5.98976994e+00,   5.83027983e+00],
        [  7.84821987e-01,   6.00147009e+00,   5.83026981e+00]], dtype=float32),
 '3': array([[  8.66532981e-01,  -1.49418996e-03,  -3.55257004e-01],
        [  8.66042972e-01,   1.02049997e-02,  -3.55253994e-01],
        [  8.65388989e-01, 

In [6]:
print(np.logspace(-1,1,40))
print(10**np.linspace(-1, 1, 100))

[  0.1          0.11253356   0.12663802   0.14251027   0.16037187
   0.18047218   0.20309176   0.22854639   0.25719138   0.28942661
   0.32570207   0.36652412   0.41246264   0.46415888   0.52233451
   0.58780161   0.66147406   0.7443803    0.83767764   0.94266846
   1.06081836   1.19377664   1.34339933   1.51177507   1.70125428
   1.91448198   2.15443469   2.42446202   2.72833338   3.07029063
   3.45510729   3.88815518   4.37547938   4.92388263   5.54102033
   6.23550734   7.01703829   7.89652287   8.88623816  10.        ]
[  0.1          0.10476158   0.10974988   0.1149757    0.12045035
   0.12618569   0.13219411   0.13848864   0.14508288   0.15199111
   0.15922828   0.16681005   0.17475284   0.18307383   0.19179103
   0.2009233    0.21049041   0.22051307   0.23101297   0.24201283
   0.25353645   0.26560878   0.27825594   0.29150531   0.30538555
   0.31992671   0.33516027   0.35111917   0.36783798   0.38535286
   0.40370173   0.42292429   0.44306215   0.46415888   0.48626016
   0.5094

In [7]:
from sklearn.neighbors.kde import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut

x = atlas_layer_points['1']
grid = GridSearchCV(KernelDensity(), #(kernel='epanechnikov'),
                    {'bandwidth': np.logspace(-1,1,10)},
                    cv=3, verbose=100, n_jobs=3,
                    return_train_score=False)
grid.fit(x);

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Memmaping (shape=(262144, 3), dtype=float32) to new file C:\Users\Ja\AppData\Local\Temp\joblib_memmaping_pool_20276_2517037656832\20276-2517037809904-9dba5fded4fff9fad85b4101fe561b16.pkl
Pickling array (shape=(174762,), dtype=int32).
Pickling array (shape=(87382,), dtype=int32).
Memmaping (shape=(262144, 3), dtype=float32) to old file C:\Users\Ja\AppData\Local\Temp\joblib_memmaping_pool_20276_2517037656832\20276-2517037809904-9dba5fded4fff9fad85b4101fe561b16.pkl
Pickling array (shape=(174763,), dtype=int32).
Pickling array (shape=(87381,), dtype=int32).
Memmaping (shape=(262144, 3), dtype=float32) to old file C:\Users\Ja\AppData\Local\Temp\joblib_memmaping_pool_20276_2517037656832\20276-2517037809904-9dba5fded4fff9fad85b4101fe561b16.pkl
Pickling array (shape=(174763,), dtype=int32).
Pickling array (shape=(87381,), dtype=int32).
Memmaping (shape=(262144, 3), dtype=float32) to old file C:\Users\Ja\AppData\Local\Temp\joblib_memm

In [8]:
grid.best_params_

{'bandwidth': 0.10000000000000001}

In [14]:
kde = grid.best_estimator_ 
print(kde)

KernelDensity(algorithm='auto', atol=0, bandwidth=0.10000000000000001,
       breadth_first=True, kernel='gaussian', leaf_size=40,
       metric='euclidean', metric_params=None, rtol=0)


In [15]:
kde.get_params()

{'algorithm': 'auto',
 'atol': 0,
 'bandwidth': 0.10000000000000001,
 'breadth_first': True,
 'kernel': 'gaussian',
 'leaf_size': 40,
 'metric': 'euclidean',
 'metric_params': None,
 'rtol': 0}

In [8]:
from sklearn.neighbors.kde import KernelDensity
kde = KernelDensity()
kde.set_params(algorithm='auto', atol=0, bandwidth=0.1, breadth_first=True, 
               kernel='gaussian', leaf_size=40, metric='euclidean', metric_params=None, rtol=0)

KernelDensity(algorithm='auto', atol=0, bandwidth=0.1, breadth_first=True,
       kernel='gaussian', leaf_size=40, metric='euclidean',
       metric_params=None, rtol=0)

In [10]:
x = atlas_layer_points['1']
logprob = kde.score_samples(x[:10,:])
prob = np.exp(logprob)

AttributeError: 'KernelDensity' object has no attribute 'tree_'

In [11]:
(50-40)/(162.5-179)

-0.6060606060606061

1.0

In [12]:
(112-138)/(100-50)

-0.52

In [14]:
(155-127)/(250-200)

0.56